<a href="https://colab.research.google.com/github/Biometrics-UT/AutoEncoderModelsPaper/blob/main/Biometrics_AutoEncoder_Run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:

# Load the different libraries etc. 
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Import the needed things. 
from __future__ import print_function
from sklearn.metrics import classification_report
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.datasets import mnist
from keras.datasets import cifar100
from random import sample,randint
from keras.layers import Input, Cropping2D
from tensorflow.keras import Model
import matplotlib.pyplot as plt
import os
import cv2
import tqdm.notebook as tq
import random
from numpy import savetxt
from PIL import Image
import matplotlib
from numpy import loadtxt
from google.colab import files
import keras.backend as K


# Set variabls
# initialize the number of epochs and batch size
EPOCHS = 50
BS = 28

# Set some initial parameters. 
batch_size = 100

# Set image sizes
img_sizeX = 500
img_sizeY = 600
input_shape=(img_sizeY,img_sizeX,3)
input_img = Input(shape = input_shape)

# Set bools
sobel = False
laplacian = True

deep = True

# Set the paths and filenames needed. 
main_path = "/content/fgrc_dataset_train"
laplacian_one_path ='/content/LaplacianImages/Laplacian_1/'
laplacian_two_path ='/content/LaplacianImages/Laplacian_2/'
laplacian_three_path ='/content/LaplacianImages/Laplacian_3/'
sobel_one_path ='/content/SobelImages/Sobel_1/'
sobel_two_path ='/content/SobelImages/Sobel_2/'
autoEncoderName = "DeepLaplacianFilterFaces"
baseEpochs = 50
loadNameFitting = None
loadNamePrediction = None

In [ ]:
#Mount the drive
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# The mean squared error. maybe remove. 
def mean_squared_error(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)

In [5]:
# Method to get Image given teh path and index
def getImage(path,index,all_imagesNames):
  # Get one test image for the 
  path = os.path.join(path, all_imagesNames[index])

  img = cv2.imread(path)
  RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  rescaled = cv2.resize(RGB_img, (img_sizeX, img_sizeY))
  rescaled = rescaled.astype('float32') / 255
  return rescaled

# Method that gets the numpy data. 
def getNpy(path, index, all_imagesNames):    
    # load array
    name = all_imagesNames[index]
    imageName = name.split('.')[0]
    path = os.path.join(path, imageName )
    data = np.load(path + ".npy")
    return data

# Methot that gets all the image names. 
def loadAllImageNames():

  # Get all the image names. 
  all_imagesNames = []
  for i in tq.tqdm(os.listdir(main_path)):
      all_imagesNames.append(i)

  # Shuffle the list for better results.
  random.shuffle(all_imagesNames)

  # Return the image names.
  return all_imagesNames

# Get a test image.
def getTestImage(imageNames):
  # Get one test image for the laplacian layers.  
  kernelTestImage = []

  # Get the image.
  image = getImage(main_path,0,imageNames)

  # Append it to the list.
  kernelTestImage.append(np.array(image))

  # Make it a workable numpy array.
  kernelTestImage = np.reshape(kernelTestImage, (len(kernelTestImage), img_sizeY, img_sizeX, 3))

  # Return the test image
  return kernelTestImage

In [6]:
# The different kernels that can be called.
def laplacian_kernel_one(shape, dtype=None):
    
    # Set the kernel.
    kernel = [[ 0.000005074932907,  0.000176841611374,  0.001371042851508, 0.00265121170289 ,  0.001371042851508,  0.000176841611374, 0.000005074932907],
		[ 0.000176841611374,  0.005040677876601,  0.029076377830313, 0.046436430819499,  0.029076377830313,  0.005040677876601, 0.000176841611374],
		[ 0.001371042851508,  0.029076377830313,  0.058641835066605, -0.049811044772298,  0.058641835066605,  0.029076377830313, 0.001371042851508],
		[ 0.00265121170289 ,  0.046436430819499, -0.049811044772298, -0.497359197162173, -0.049811044772298,  0.046436430819499, 0.00265121170289 ],
		[ 0.001371042851508,  0.029076377830313,  0.058641835066605, -0.049811044772298,  0.058641835066605,  0.029076377830313, 0.001371042851508],
		[ 0.000176841611374,  0.005040677876601,  0.029076377830313, 0.046436430819499,  0.029076377830313,  0.005040677876601, 0.000176841611374],
		[ 0.000005074932907,  0.000176841611374,  0.001371042851508, 0.00265121170289 ,  0.001371042851508,  0.000176841611374, 0.000005074932907]]
    
    # Create the 3D filter.
    filters = tf.transpose(tf.constant([[kernel, kernel, kernel]], dtype=tf.float32), (2, 3, 1, 0))
  
    # Return the filter.
    return filters

def laplacian_kernel_two(shape, dtype=None):
    
    # Set the kernel.
    kernel = [	
		[ 0.000001268733227,  0.000009086256607,  0.000044210402843, 0.000147589005316,  0.000342760712877,  0.000562862476099, 0.000662802925722,  0.000562862476099,  0.000342760712877, 0.000147589005316,  0.000044210402843,  0.000009086256607, 0.000001268733227],
		[ 0.000009086256607,  0.000062551250942,  0.000290030622363, 0.000916116269692,  0.002011423105258,  0.003159868644707, 0.003657447123561,  0.003159868644707,  0.002011423105258, 0.000916116269692,  0.000290030622363,  0.000062551250942, 0.000009086256607],
		[ 0.000044210402843,  0.000290030622363,  0.00126016946915, 0.003657447123561,  0.007269094457578,  0.01042709294374 , 0.011609107704875,  0.01042709294374 ,  0.007269094457578, 0.003657447123561,  0.00126016946915 ,  0.000290030622363, 0.000044210402843],
		[ 0.000147589005316,  0.000916116269692,  0.003657447123561, 0.009299070196277,  0.015106560194771,  0.016808486630785, 0.016246643753945,  0.016808486630785,  0.015106560194771, 0.009299070196277,  0.003657447123561,  0.000916116269692, 0.000147589005316],
		[ 0.000342760712877,  0.002011423105258,  0.007269094457578, 0.015106560194771,  0.014660458766651, -0.001097503065786, -0.012452761193074, -0.001097503065786,  0.014660458766651, 0.015106560194771,  0.007269094457578,  0.002011423105258, 0.000342760712877],
		[ 0.000562862476099,  0.003159868644707,  0.01042709294374 , 0.016808486630785, -0.001097503065786, -0.051268252318321, -0.082302736624876, -0.051268252318321, -0.001097503065786, 0.016808486630785,  0.01042709294374 ,  0.003159868644707, 0.000562862476099],
		[ 0.000662802925722,  0.003657447123561,  0.011609107704875, 0.016246643753945, -0.012452761193074, -0.082302736624876, -0.124339799290543, -0.082302736624876, -0.012452761193074, 0.016246643753945,  0.011609107704875,  0.003657447123561, 0.000662802925722],
		[ 0.000562862476099,  0.003159868644707,  0.01042709294374 , 0.016808486630785, -0.001097503065786, -0.051268252318321, -0.082302736624876, -0.051268252318321, -0.001097503065786, 0.016808486630785,  0.01042709294374 ,  0.003159868644707, 0.000562862476099],
		[ 0.000342760712877,  0.002011423105258,  0.007269094457578, 0.015106560194771,  0.014660458766651, -0.001097503065786, -0.012452761193074, -0.001097503065786,  0.014660458766651, 0.015106560194771,  0.007269094457578,  0.002011423105258, 0.000342760712877],
		[ 0.000147589005316,  0.000916116269692,  0.003657447123561, 0.009299070196277,  0.015106560194771,  0.016808486630785, 0.016246643753945,  0.016808486630785,  0.015106560194771, 0.009299070196277,  0.003657447123561,  0.000916116269692, 0.000147589005316],
		[ 0.000044210402843,  0.000290030622363,  0.00126016946915, 0.003657447123561,  0.007269094457578,  0.01042709294374, 0.011609107704875,  0.01042709294374 ,  0.007269094457578, 0.003657447123561,  0.00126016946915 ,  0.000290030622363, 0.000044210402843],
		[ 0.000009086256607,  0.000062551250942,  0.000290030622363, 0.000916116269692,  0.002011423105258,  0.003159868644707, 0.003657447123561,  0.003159868644707,  0.002011423105258, 0.000916116269692,  0.000290030622363,  0.000062551250942, 0.000009086256607],
		[ 0.000001268733227,  0.000009086256607,  0.000044210402843, 0.000147589005316,  0.000342760712877,  0.000562862476099, 0.000662802925722,  0.000562862476099,  0.000342760712877, 0.000147589005316,  0.000044210402843,  0.000009086256607, 0.000001268733227]
	]
		
    # Create the 3D filter.
    filters = tf.transpose(tf.constant([[kernel, kernel, kernel]], dtype=tf.float32), (2, 3, 1, 0))
  
    # Return the filter.
    return filters
    
def laplacian_kernel_three(shape, dtype=None):
    
    # Set the kernel.
    kernel =[	
		[ 0.00000003276694 ,  0.000000102321053,  0.000000289823431,0.000000745118474,  0.000001740181075,  0.000003695490265,0.000007144471547,  0.000012591506332,  0.00002026037218 ,0.000029810457975,  0.000040172249113,  0.000049653076997,0.000056355461988,  0.000058779711723,  0.000056355461988,0.000049653076997,  0.000040172249113,  0.000029810457975,0.00002026037218 ,  0.000012591506332,  0.000007144471547,0.000003695490265,  0.000001740181075,  0.000000745118474,0.000000289823431,  0.000000102321053,  0.00000003276694 ],
		[ 0.000000102321053,  0.000000317183307,  0.000000891254046,0.000002271564152,  0.000005255987472,  0.000011052600711,0.000021151594635,  0.000036897251329,  0.000058779711723,0.000085690178219,  0.00011455564698 ,  0.000140715619025,0.000159081790068,  0.000165700731431,  0.000159081790068,0.000140715619025,  0.00011455564698 ,  0.000085690178219,0.000058779711723,  0.000036897251329,  0.000021151594635,0.000011052600711,  0.000005255987472,  0.000002271564152,0.000000891254046,  0.000000317183307,  0.000000102321053],
		[ 0.000000289823431,  0.000000891254046,  0.000002482179193,0.000006264686954,  0.000014340889217,  0.000029810457975,0.000056355461988,  0.000097075182089,  0.000152717393296,0.000220002752893,  0.000291045957226,  0.000354565490002,0.000398718169712,  0.000414545956269,  0.000398718169712,0.000354565490002,  0.000291045957226,  0.000220002752893,0.000152717393296,  0.000097075182089,  0.000056355461988,0.000029810457975,  0.000014340889217,  0.000006264686954, 0.000002482179193,  0.000000891254046,  0.000000289823431],
		[ 0.000000745118474,  0.000002271564152,  0.000006264686954,0.000015637812736,  0.00003535918443 ,  0.000072507655591,0.000135060680357,  0.000229029067423,  0.000354565490002,0.000502855776314,  0.000655890033161,  0.000789967161177,0.000881738236641,  0.00091436178089 ,  0.000881738236641,0.000789967161177,  0.000655890033161,  0.000502855776314,0.000354565490002,  0.000229029067423,  0.000135060680357,0.000072507655591,  0.00003535918443 ,  0.000015637812736, 0.000006264686954,  0.000002271564152,  0.000000745118474],
		[ 0.000001740181075,  0.000005255987472,  0.000014340889217,0.00003535918443 ,  0.000078831688329,  0.000159081790068,0.000291045957226,  0.000483905610509,  0.000733625931829,0.001018671159288,  0.001302594707778,  0.001543202189799,0.001703578516854,  0.00175974821168 ,  0.001703578516854,0.001543202189799,  0.001302594707778,  0.001018671159288,0.000733625931829,  0.000483905610509,  0.000291045957226,0.000159081790068,  0.000078831688329,  0.00003535918443 ,0.000014340889217,  0.000005255987472,  0.000001740181075],
		[ 0.000003695490265,  0.000011052600711,  0.000029810457975,0.000072507655591,  0.000159081790068,  0.000315042367288,0.000563900019106,  0.00091436178089 ,  0.001348101790401,0.001817273614395,  0.002256953506164,  0.002606773235935,0.002827478640437,  0.002902276926219,  0.002827478640437,0.002606773235935,  0.002256953506164,  0.001817273614395,0.001348101790401,  0.00091436178089 ,  0.000563900019106,0.000315042367288,  0.000159081790068,  0.000072507655591, 0.000029810457975,  0.000011052600711,  0.000003695490265],
		[ 0.000007144471547,  0.000021151594635,  0.000056355461988,0.000135060680357,  0.000291045957226,  0.000563900019106,0.000982804254813,  0.001543202189799,  0.002190323257653,0.002827478640437,  0.003353879875579,  0.003710770856355,0.003899844665886,  0.003956273092643,  0.003899844665886,0.003710770856355,  0.003353879875579,  0.002827478640437,0.002190323257653,  0.001543202189799,  0.000982804254813,0.000563900019106,  0.000291045957226,  0.000135060680357,0.000056355461988,  0.000021151594635,  0.000007144471547],
		[ 0.000012591506332,  0.000036897251329,  0.000097075182089,0.000229029067423,  0.000483905610509,  0.00091436178089 ,0.001543202189799,  0.002324767549069,  0.003128504290034,0.003776640048693,  0.004135046026703,  0.004202121657696,0.004117335028969,  0.004061660938486,  0.004117335028969,0.004202121657696,  0.004135046026703,  0.003776640048693,0.003128504290034,  0.002324767549069,  0.001543202189799,0.00091436178089 ,  0.000483905610509,  0.000229029067423, 0.000097075182089,  0.000036897251329,  0.000012591506332],
		[ 0.00002026037218 ,  0.000058779711723,  0.000152717393296,0.000354565490002,  0.000733625931829,  0.001348101790401,0.002190323257653,  0.003128504290034,  0.003899844665886,0.004206882475076,  0.003901217929368,  0.003138282542174,0.002354013123118,  0.002024016375783,  0.002354013123118,0.003138282542174,  0.003901217929368,  0.004206882475076,0.003899844665886,  0.003128504290034,  0.002190323257653,0.001348101790401,  0.000733625931829,  0.000354565490002,0.000152717393296,  0.000058779711723,  0.00002026037218 ],
		[ 0.000029810457975,  0.000085690178219,  0.000220002752893,0.000502855776314,  0.001018671159288,  0.001817273614395,0.002827478640437,  0.003776640048693,  0.004206882475076,0.003665114691663,  0.002024016375783, -0.000274375766446,-0.002303038048863, -0.003113190298269, -0.002303038048863,-0.000274375766446,  0.002024016375783,  0.003665114691663,0.004206882475076,  0.003776640048693,  0.002827478640437,0.001817273614395,  0.001018671159288,  0.000502855776314,0.000220002752893,  0.000085690178219,  0.000029810457975],
		[ 0.000040172249113,  0.00011455564698 ,  0.000291045957226,0.000655890033161,  0.001302594707778,  0.002256953506164,0.003353879875579,  0.004135046026703,  0.003901217929368,0.002024016375783, -0.001563031623073, -0.006017922742186,-0.009761659358786, -0.011228197398305, -0.009761659358786,-0.006017922742186, -0.001563031623073,  0.002024016375783,0.003901217929368,  0.004135046026703,  0.003353879875579,0.002256953506164,  0.001302594707778,  0.000655890033161,0.000291045957226,  0.00011455564698 ,  0.000040172249113],
		[ 0.000049653076997,  0.000140715619025,  0.000354565490002,0.000789967161177,  0.001543202189799,  0.002606773235935,0.003710770856355,  0.004202121657696,  0.003138282542174,-0.000274375766446, -0.006017922742186, -0.01281706307958 ,-0.018406119972447, -0.020575684156219, -0.018406119972447,-0.01281706307958 , -0.006017922742186, -0.000274375766446,0.003138282542174,  0.004202121657696,  0.003710770856355,0.002606773235935,  0.001543202189799,  0.000789967161177,0.000354565490002,  0.000140715619025,  0.000049653076997],
		[ 0.000056355461988,  0.000159081790068,  0.000398718169712,0.000881738236641,  0.001703578516854,  0.002827478640437,0.003899844665886,  0.004117335028969,  0.002354013123118,-0.002303038048863, -0.009761659358786, -0.018406119972447,-0.025439619702615, -0.028158102510065, -0.025439619702615,-0.018406119972447, -0.009761659358786, -0.002303038048863,0.002354013123118,  0.004117335028969,  0.003899844665886,0.002827478640437,  0.001703578516854,  0.000881738236641,0.000398718169712,  0.000159081790068,  0.000056355461988],
		[ 0.000058779711723,  0.000165700731431,  0.000414545956269,0.00091436178089 ,  0.00175974821168 ,  0.002902276926219,0.003956273092643,  0.004061660938486,  0.002024016375783,-0.003113190298269, -0.011228197398305, -0.020575684156219,-0.028158102510065, -0.031084949822636, -0.028158102510065,-0.020575684156219, -0.011228197398305, -0.003113190298269,0.002024016375783,  0.004061660938486,  0.003956273092643,0.002902276926219,  0.00175974821168 ,  0.00091436178089 ,0.000414545956269,  0.000165700731431,  0.000058779711723],
		[ 0.000056355461988,  0.000159081790068,  0.000398718169712,0.000881738236641,  0.001703578516854,  0.002827478640437,0.003899844665886,  0.004117335028969,  0.002354013123118,-0.002303038048863, -0.009761659358786, -0.018406119972447,-0.025439619702615, -0.028158102510065, -0.025439619702615,-0.018406119972447, -0.009761659358786, -0.002303038048863,0.002354013123118,  0.004117335028969,  0.003899844665886,0.002827478640437,  0.001703578516854,  0.000881738236641,0.000398718169712,  0.000159081790068,  0.000056355461988],
		[ 0.000049653076997,  0.000140715619025,  0.000354565490002,0.000789967161177,  0.001543202189799,  0.002606773235935,0.003710770856355,  0.004202121657696,  0.003138282542174,-0.000274375766446, -0.006017922742186, -0.01281706307958 ,-0.018406119972447, -0.020575684156219, -0.018406119972447,-0.01281706307958 , -0.006017922742186, -0.000274375766446,0.003138282542174,  0.004202121657696,  0.003710770856355,0.002606773235935,  0.001543202189799,  0.000789967161177,0.000354565490002,  0.000140715619025,  0.000049653076997],
		[ 0.000040172249113,  0.00011455564698 ,  0.000291045957226,0.000655890033161,  0.001302594707778,  0.002256953506164,0.003353879875579,  0.004135046026703,  0.003901217929368,0.002024016375783, -0.001563031623073, -0.006017922742186,-0.009761659358786, -0.011228197398305, -0.009761659358786,-0.006017922742186, -0.001563031623073,  0.002024016375783,0.003901217929368,  0.004135046026703,  0.003353879875579,0.002256953506164,  0.001302594707778,  0.000655890033161,0.000291045957226,  0.00011455564698 ,  0.000040172249113],
		[ 0.000029810457975,  0.000085690178219,  0.000220002752893,0.000502855776314,  0.001018671159288,  0.001817273614395,0.002827478640437,  0.003776640048693,  0.004206882475076,0.003665114691663,  0.002024016375783, -0.000274375766446,-0.002303038048863, -0.003113190298269, -0.002303038048863,-0.000274375766446,  0.002024016375783,  0.003665114691663,0.004206882475076,  0.003776640048693,  0.002827478640437,0.001817273614395,  0.001018671159288,  0.000502855776314,0.000220002752893,  0.000085690178219,  0.000029810457975],
		[ 0.00002026037218 ,  0.000058779711723,  0.000152717393296,0.000354565490002,  0.000733625931829,  0.001348101790401,0.002190323257653,  0.003128504290034,  0.003899844665886,0.004206882475076,  0.003901217929368,  0.003138282542174,0.002354013123118,  0.002024016375783,  0.002354013123118,0.003138282542174,  0.003901217929368,  0.004206882475076,0.003899844665886,  0.003128504290034,  0.002190323257653,0.001348101790401,  0.000733625931829,  0.000354565490002,0.000152717393296,  0.000058779711723,  0.00002026037218 ],
		[ 0.000012591506332,  0.000036897251329,  0.000097075182089,0.000229029067423,  0.000483905610509,  0.00091436178089 ,0.001543202189799,  0.002324767549069,  0.003128504290034,0.003776640048693,  0.004135046026703,  0.004202121657696,0.004117335028969,  0.004061660938486,  0.004117335028969,0.004202121657696,  0.004135046026703,  0.003776640048693,0.003128504290034,  0.002324767549069,  0.001543202189799,0.00091436178089 ,  0.000483905610509,  0.000229029067423,0.000097075182089,  0.000036897251329,  0.000012591506332],
		[ 0.000007144471547,  0.000021151594635,  0.000056355461988,0.000135060680357,  0.000291045957226,  0.000563900019106,0.000982804254813,  0.001543202189799,  0.002190323257653,0.002827478640437,  0.003353879875579,  0.003710770856355,0.003899844665886,  0.003956273092643,  0.003899844665886,0.003710770856355,  0.003353879875579,  0.002827478640437,0.002190323257653,  0.001543202189799,  0.000982804254813,0.000563900019106,  0.000291045957226,  0.000135060680357,0.000056355461988,  0.000021151594635,  0.000007144471547],
		[ 0.000003695490265,  0.000011052600711,  0.000029810457975,0.000072507655591,  0.000159081790068,  0.000315042367288,0.000563900019106,  0.00091436178089 ,  0.001348101790401,0.001817273614395,  0.002256953506164,  0.002606773235935,0.002827478640437,  0.002902276926219,  0.002827478640437,0.002606773235935,  0.002256953506164,  0.001817273614395,0.001348101790401,  0.00091436178089 ,  0.000563900019106,0.000315042367288,  0.000159081790068,  0.000072507655591,0.000029810457975,  0.000011052600711,  0.000003695490265],
		[ 0.000001740181075,  0.000005255987472,  0.000014340889217,0.00003535918443 ,  0.000078831688329,  0.000159081790068,0.000291045957226,  0.000483905610509,  0.000733625931829,0.001018671159288,  0.001302594707778,  0.001543202189799,0.001703578516854,  0.00175974821168 ,  0.001703578516854,0.001543202189799,  0.001302594707778,  0.001018671159288,0.000733625931829,  0.000483905610509,  0.000291045957226,0.000159081790068,  0.000078831688329,  0.00003535918443 ,0.000014340889217,  0.000005255987472,  0.000001740181075],
		[ 0.000000745118474,  0.000002271564152,  0.000006264686954,0.000015637812736,  0.00003535918443 ,  0.000072507655591,0.000135060680357,  0.000229029067423,  0.000354565490002,0.000502855776314,  0.000655890033161,  0.000789967161177,0.000881738236641,  0.00091436178089 ,  0.000881738236641,0.000789967161177,  0.000655890033161,  0.000502855776314,0.000354565490002,  0.000229029067423,  0.000135060680357,0.000072507655591,  0.00003535918443 ,  0.000015637812736,0.000006264686954,  0.000002271564152,  0.000000745118474],
		[ 0.000000289823431,  0.000000891254046,  0.000002482179193,0.000006264686954,  0.000014340889217,  0.000029810457975,0.000056355461988,  0.000097075182089,  0.000152717393296,0.000220002752893,  0.000291045957226,  0.000354565490002,0.000398718169712,  0.000414545956269,  0.000398718169712,0.000354565490002,  0.000291045957226,  0.000220002752893,0.000152717393296,  0.000097075182089,  0.000056355461988,0.000029810457975,  0.000014340889217,  0.000006264686954,0.000002482179193,  0.000000891254046,  0.000000289823431],
		[ 0.000000102321053,  0.000000317183307,  0.000000891254046,0.000002271564152,  0.000005255987472,  0.000011052600711,0.000021151594635,  0.000036897251329,  0.000058779711723,0.000085690178219,  0.00011455564698 ,  0.000140715619025,0.000159081790068,  0.000165700731431,  0.000159081790068,0.000140715619025,  0.00011455564698 ,  0.000085690178219,0.000058779711723,  0.000036897251329,  0.000021151594635,0.000011052600711,  0.000005255987472,  0.000002271564152,0.000000891254046,  0.000000317183307,  0.000000102321053],
		[ 0.00000003276694 ,  0.000000102321053,  0.000000289823431,0.000000745118474,  0.000001740181075,  0.000003695490265,0.000007144471547,  0.000012591506332,  0.00002026037218 ,0.000029810457975,  0.000040172249113,  0.000049653076997,0.000056355461988,  0.000058779711723,  0.000056355461988,0.000049653076997,  0.000040172249113,  0.000029810457975,0.00002026037218 ,  0.000012591506332,  0.000007144471547,0.000003695490265,  0.000001740181075,  0.000000745118474,0.000000289823431,  0.000000102321053,  0.00000003276694 ]]
    
    # Return the filter.
    filters = tf.transpose(tf.constant([[kernel, kernel, kernel]], dtype=tf.float32), (2, 3, 1, 0))
  
    return filters

def sobel_kernel_one(shape, dtype=None):
    
    # Set the kernel.
    kernel = [
              [ 1,0,-1],
		          [ 2,0,-2],
		          [ 1,0,-1]
            ]
    
    # Create the 3D filter.
    filters = tf.transpose(tf.constant([[kernel, kernel, kernel]], dtype=tf.float32), (2, 3, 1, 0))
  
    # Return the filter.
    return filters

def sobel_kernel_two(shape, dtype=None):
    
    # Set the kernel.
    kernel = [	
		[1,2,1],
		[ 0,0,0],
		[ -1,-2,-1]
	  ]
		
    # Create the 3D filter.
    filters = tf.transpose(tf.constant([[kernel, kernel, kernel]], dtype=tf.float32), (2, 3, 1, 0))
  
    # Return the filter.
    return filters



In [7]:
# Method that creates a model for the different kernels depending on the parameters.
def createModel(kernelSize, kernelInitializer, saveName, kernelTestImages):
    # Set the batchize
    batchSize = 100
    
    # Set the layer. 
    layer1 =  keras.layers.Conv2D(1, kernel_size=kernelSize, kernel_initializer=kernelInitializer, padding='same', name="oneLayer")(input_img)
    createdModel = Model(input_img, layer1)
    createdModel.get_layer("oneLayer").trainable = False

    # Compile and give a summary of the model
    createdModel.compile(optimizer='adam',loss=mean_squared_error, metrics=['accuracy'])
    createdModel.summary()

    # Save the model, and save it.
    createdModel.fit(kernelTestImages, kernelTestImages, epochs =1, batch_size =batchSize, validation_data=(kernelTestImages, kernelTestImages))
    createdModel.save(saveName)

In [8]:
# Method that creates a model for the filters depending on if it is sobel or not.
def createModels(sobel, kernelTestImages):

  # If sobel arrays.
  if sobel:
    # Create the two kernels. 
    sobel = createModel((3, 3),sobel_kernel_one,"/content/drive/MyDrive/Models/sobel1_OneImage", kernelTestImages)
    sobel = createModel((3, 3),sobel_kernel_two,"/content/drive/MyDrive/Models/sobel2_OneImage", kernelTestImages)    
  else: # Else create laplacian
    laplacian = createModel((7,7),laplacian_kernel_one,"/content/drive/MyDrive/Models/laplacian1_OneImage", kernelTestImages)
    laplacian = createModel((13,13),laplacian_kernel_two,"/content/drive/MyDrive/Models/laplacian2_OneImage", kernelTestImages)
    laplacian = createModel((27,27),laplacian_kernel_three,"/content/drive/MyDrive/Models/laplacian3_OneImage", kernelTestImages)

In [10]:
# Method that loads models. Sobel parameter indicator if it is sobel or laplacian
def loadModels(sobel):

  # Array to hold the models.
  modelArray = []

  # If sobel, load the sobel arrays, else load laplacianmodels.
  if sobel:
    modelArray.append(load_model("/content/drive/MyDrive/Models/sobel1_OneImage"))
    modelArray.append(load_model("/content/drive/MyDrive/Models/sobel2_OneImage"))
  else:
    modelArray.append(load_model("/content/drive/MyDrive/Models/laplacian1_OneImage"))
    modelArray.append(load_model("/content/drive/MyDrive/Models/laplacian2_OneImage"))
    modelArray.append(load_model("/content/drive/MyDrive/Models/laplacian3_OneImage"))

  return modelArray

In [9]:
# Method that creates the nuumpy arrays based on where to save it and which models need to be done.
def createSaveNumpyArrays(locationList, all_imagesNames, main_path, models):

  # Create the directories if they don't exist.
  for location in locationList:
    if not os.path.exists(location):
      os.makedirs(location)

  # Get the laplacian images
  for index in  tq.tqdm(range(len(all_imagesNames))):

    # Get the image name. 
    imageName = all_imagesNames[index].split('.')[0]

    # Get an image and reshape it so it can be used. 
    image = getImage(main_path,index,all_imagesNames)

    # Reform the acutal image to work on.
    actualImage = np.reshape([image], (1, img_sizeY, img_sizeX, 3))

    # Loop over the different models. 
    for x in range(len(models)):
      
      # Get the prediction of the image.
      modelledImage = models[x].predict(actualImage)

      # Save the image. 
      np.save(locationList[x] +'/{}'.format(imageName), modelledImage[0])

In [11]:
# Method that gets the paper sized auto encoder and adds sobel or laplacian layers if need be
def getPaper(sobel, laplacian):

  # Encoder
  conv1 =  keras.layers.Conv2D(32, (3, 3), activation='relu',padding='same')(input_img) # layer 2

  # Layer 3
  pool1 = keras.layers.MaxPool2D(pool_size=(2, 2))(conv1) 
  conv2 = keras.layers.Conv2D(16, (3, 3), activation='relu',padding='same')(pool1)

  #decoder 

  #layer 4
  up1 = keras.layers.UpSampling2D((2,2))(conv2)
  conv3 = keras.layers.Conv2D(32, (3, 3), activation='relu',padding='same')(up1)
  #layer 5
  decoded = keras.layers.Conv2D(3, (3, 3), activation='tanh',padding='same', name = "decoded")(conv3)

  if sobel:
    # The sobel branching layers. 
    sobel1Actual =  keras.layers.Conv2D(1, kernel_size=(3, 3), kernel_initializer=sobel_kernel_one, padding='same', name="sobel1")(decoded)
    sobel2Actual =  keras.layers.Conv2D(1, kernel_size=(3, 3), kernel_initializer=sobel_kernel_two, padding='same', name="sobel2")(decoded)

    # Define the different outputs.
    autoencoder = Model(input_img, outputs = [decoded,sobel1Actual,sobel2Actual])

    # Set the layers to not train. 
    autoencoder.get_layer("sobel1").trainable = False
    autoencoder.get_layer("sobel2").trainable = False

    # Set the weights array. 
    lossWeight = [1/2.0,1/4.0,1/4.0]

    # Compile
    autoencoder.compile(optimizer='adam',loss=[mean_squared_error, mean_squared_error, mean_squared_error], metrics=['accuracy'],loss_weights=lossWeight)
    autoencoder.summary()

    # Return the auto encoder.
    return autoencoder

  elif laplacian:
    # The laplacian branching layers. 
    laplacian1Actual =  keras.layers.Conv2D(1, kernel_size=(7, 7), kernel_initializer=laplacian_kernel_one, padding='same', name="laplacian1")(decoded)
    laplacian2Actual =  keras.layers.Conv2D(1, kernel_size=(13, 13), kernel_initializer=laplacian_kernel_two, padding='same', name="laplacian2")(decoded)
    laplacian3Actual =  keras.layers.Conv2D(1, kernel_size=(27, 27), kernel_initializer=laplacian_kernel_three, padding='same', name="laplacian3")(decoded)

    # Define the different outputs.
    autoencoder = Model(input_img, outputs = [decoded,laplacian1Actual,laplacian2Actual,laplacian3Actual])

    # Set the layers to not train. 
    autoencoder.get_layer("laplacian1").trainable = False
    autoencoder.get_layer("laplacian2").trainable = False
    autoencoder.get_layer("laplacian3").trainable = False

    # Set the weights array. 
    lossWeight = [1/2.0,1/6.0,1/6.0,1/6.0]

    # Compile
    autoencoder.compile(optimizer='adam',loss=[mean_squared_error, mean_squared_error,mean_squared_error,mean_squared_error], metrics=['accuracy'],loss_weights=lossWeight)
    autoencoder.summary()

    # Return the auto encoder.
    return autoencoder

  else: # Normal auto encoder
    # Set the values for one it has no filters. 
    autoencoder = Model(input_img, outputs = decoded)

    # Compile
    autoencoder.compile(optimizer='adam',loss=mean_squared_error, metrics=['accuracy'])
    autoencoder.summary()

    # Return the auto encoder.
    return autoencoder

# Method that gets the deeper auto encoder and adds the laplacian or sobel layer if needed. 
def getDeep(sobel, laplacian):

  # THe layers of the algorithm, these ones are for the paper.
  conv1 =  keras.layers.Conv2D(32, (3, 3), activation='relu',padding='same')(input_img) # layer 2
  pool1 = keras.layers.MaxPool2D(pool_size=(2, 2), padding= 'same')(conv1) # next two lines are layer 3
  conv2 = keras.layers.Conv2D(16, (3, 3), activation='relu',padding='same')(pool1)

  pool2 = keras.layers.MaxPool2D(pool_size=(2, 2),padding= "same")(conv2) # next two lines are an extra layer
  conv3 = keras.layers.Conv2D(8, (3, 3), activation='relu',padding='same')(pool2)

  pool3 = keras.layers.MaxPool2D(pool_size=(2, 2),padding= "same")(conv3) # next two lines are another extra layer
  conv4 = keras.layers.Conv2D(4, (3, 3), activation='relu',padding='same')(pool3)

  pool4 = keras.layers.MaxPool2D(pool_size=(2, 2),padding= "same")(conv4) # next two lines are another extra layer
  conv5 = keras.layers.Conv2D(2, (3, 3), activation='relu',padding='same')(pool4)

  #decoder 

  # Extra layer to combat one above
  up1 = keras.layers.UpSampling2D((2,2))(conv5)
  conv6 = keras.layers.Conv2D(4, (3, 3), activation='relu',padding='same')(up1)

  # Crop to get the correct size.
  cropped = Cropping2D(cropping=((1, 0), (1, 0)), data_format=None)(conv6)


  # Extra layer to combat one above
  up2 = keras.layers.UpSampling2D((2,2))(cropped)
  conv7 = keras.layers.Conv2D(8, (3, 3), activation='relu',padding='same')(up2)

  # Crop to get the correct size.
  cropped = Cropping2D(cropping=((0, 0), (1, 0)), data_format=None)(conv7) 

  # Extra layer to combat one above
  up3 = keras.layers.UpSampling2D((2,2))(cropped)
  conv8 = keras.layers.Conv2D(16, (3, 3), activation='relu',padding='same')(up3)

  #layer 4
  up4 = keras.layers.UpSampling2D((2,2))(conv8)
  conv9 = keras.layers.Conv2D(32, (3, 3), activation='relu',padding='same')(up4)
  #layer 5
  decoded = keras.layers.Conv2D(3, (3, 3), activation='tanh',padding='same', name = "decoded")(conv9)

  if sobel:
    # The sobel branching layers. 
    sobel1Actual =  keras.layers.Conv2D(1, kernel_size=(3, 3), kernel_initializer=sobel_kernel_one, padding='same', name="sobel1")(decoded)
    sobel2Actual =  keras.layers.Conv2D(1, kernel_size=(3, 3), kernel_initializer=sobel_kernel_two, padding='same', name="sobel2")(decoded)

    # Define the different outputs.
    autoencoder = Model(input_img, outputs = [decoded,sobel1Actual,sobel2Actual])

    # Set the layers to not train. 
    autoencoder.get_layer("sobel1").trainable = False
    autoencoder.get_layer("sobel2").trainable = False

    # Set the weights array. 
    lossWeight = [1/2.0,1/4.0,1/4.0]

    # Compile
    autoencoder.compile(optimizer='adam',loss=[mean_squared_error, mean_squared_error, mean_squared_error], metrics=['accuracy'],loss_weights=lossWeight)
    autoencoder.summary()

    # Return the auto encoder.
    return autoencoder

  elif laplacian:
    # The laplacian branching layers. 
    laplacian1Actual =  keras.layers.Conv2D(1, kernel_size=(7, 7), kernel_initializer=laplacian_kernel_one, padding='same', name="laplacian1")(decoded)
    laplacian2Actual =  keras.layers.Conv2D(1, kernel_size=(13, 13), kernel_initializer=laplacian_kernel_two, padding='same', name="laplacian2")(decoded)
    laplacian3Actual =  keras.layers.Conv2D(1, kernel_size=(27, 27), kernel_initializer=laplacian_kernel_three, padding='same', name="laplacian3")(decoded)

    # Define the different outputs.
    autoencoder = Model(input_img, outputs = [decoded,laplacian1Actual,laplacian2Actual,laplacian3Actual])

    # Set the layers to not train. 
    autoencoder.get_layer("laplacian1").trainable = False
    autoencoder.get_layer("laplacian2").trainable = False
    autoencoder.get_layer("laplacian3").trainable = False

    # Set the weights array. 
    lossWeight = [1/2.0,1/6.0,1/6.0,1/6.0]

    # Compile.
    autoencoder.compile(optimizer='adam',loss=[mean_squared_error, mean_squared_error,mean_squared_error,mean_squared_error], metrics=['accuracy'],loss_weights=lossWeight)
    autoencoder.summary()

    # Return the auto encoder.
    return autoencoder

  else:
    # Set the values for one it has no filters. 
    autoencoder = Model(input_img, outputs = decoded)

    # Compile.
    autoencoder.compile(optimizer='adam',loss=mean_squared_error, metrics=['accuracy'])
    autoencoder.summary()

    # Return the auto encoder.
    return autoencoder

In [12]:
# Method that gets the images for the training. 
def image_generator(bs, startPos, endPos, sobel, laplacian, locationList, imageNameList ):
    # Set the index
    index = startPos

    while True:
        # Initialize our image batches.
        startImages = []
        imageLibrary = {}
        for x in range(len(locationList)):
          imageLibrary[x] = []
        # keep looping until we reach our batch size
        while (len(startImages) < bs):
            if(index == endPos):
                index = startPos

            # Get the images and numpy array.s 
            startImages.append(np.array(getImage(main_path, index,imageNameList)))
            for x in range(len(locationList)):
              imageLibrary[x].append((getNpy(locationList[x], index, imageNameList)))

            # Increment the index. 
            index +=1

        # Reshape the arrays
        startImages = np.reshape(startImages, (len(startImages), img_sizeY, img_sizeX, 3))
        
        for x in range(len(locationList)):
          imageLibrary[x] = np.reshape(imageLibrary[x], (len(imageLibrary[x]), img_sizeY, img_sizeX, 1)) 

        # Yield the images. 
        if laplacian:
          yield startImages, { "decoded" : startImages, "laplacian1" : imageLibrary[0], "laplacian2" : imageLibrary[1], "laplacian3" :imageLibrary[2]}
        elif sobel:
          # Yield the images. 
          yield startImages, { "decoded" : startImages, "sobel1" : imageLibrary[0], "sobel2" : imageLibrary[1]}
        else:
          yield startImages, { "decoded" : startImages} 

In [13]:
# Method that returns teh callback.
def getCallbacks(filepath):

  # Create the callback function
  checkpoint_filepath = filepath + "-{epoch:03d}.h5"
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath,
      save_weights_only=False,
      monitor='loss',
      verbose = 1,
      mode='min',
      save_best_only = True,
      epochs = 10
    )

  # Return the callbacks. 
  return [model_checkpoint_callback]

In [14]:
# Method that saves losses based on the fitted results, name of model, epochs ran and if there is a filter used or not.
def saveLosses(fitted, modelName, epochCount, filter):

  # Set the save location names. 
  lossHistoryName = "/content/drive/MyDrive/Loss/" + modelName + "lossHistoryEpoch" + str(epochCount) + ".txt"
  vallHistoryName = "/content/drive/MyDrive/Loss/" + modelName + "vall_lossHistoryEpoch" + str(epochCount) + ".txt"
  decodedHistoryName = "/content/drive/MyDrive/Loss/" + modelName + "decoded_lossHistoryEpoch" + str(epochCount) + ".txt"

  # Get the numpy loss information.
  loss_history = fitted.history["loss"]
  numpy_loss_history = np.array(loss_history)
  np.savetxt(lossHistoryName, numpy_loss_history, delimiter=",")
  files.download(lossHistoryName)

  # Get the numpy vallloss information.
  loss_history = fitted.history["val_loss"]
  numpy_loss_history = np.array(loss_history)
  np.savetxt(vallHistoryName, numpy_loss_history, delimiter=",")
  files.download(vallHistoryName)

  # If there are no filters, we are done now.
  if not filter:
    return

  # Get the numpy loss information.
  loss_history = fitted.history["decoded_loss"]
  numpy_loss_history = np.array(loss_history)
  np.savetxt(decodedHistoryName, numpy_loss_history, delimiter=",")
  files.download(decodedHistoryName)

In [17]:
# Method that does predictions on all the images and saves them
def getPrediction(autoEncoder,main_path, all_imagesNames, filters ):
  # Make the folder if it doesn't exist. 
  if not os.path.exists('Predicted/'):
      os.makedirs('Predicted/')

  # Loop over the incdices.
  for index in tq.tqdm(range(len(all_imagesNames))):

      # Get an immage.
      image = getImage(main_path, index, all_imagesNames)

      # Create a list to work on.
      predictImage = [(np.array(image))]

      # Make it a workable numpy array.
      predictImage = np.reshape(predictImage, (len(predictImage), img_sizeY, img_sizeX, 3))

      # Get the all the images it creates. 
      returnedImageList =  autoEncoder.predict(predictImage)

      # Get the image name.
      imageName = all_imagesNames[index].split('.')[0]

      # Set teh variable. 
      returnedImage = None

      # If filters are used we need to grab the image differently.
      if filters:
        # Save the actual first image in the list, grab the first one in the "list" and then the decoded one.
        returnedImage = returnedImageList[0][0]
      else:
        returnedImage = returnedImageList[0]

      # Clip the image values. 
      returnedImage = np.clip(returnedImage,0,1)
      
      # Save the image. 
      matplotlib.image.imsave("Predicted/{}.png".format(imageName), returnedImage)
   
    

In [16]:
# Method that displays the decoded image, and the original
def displayImage(decoded_imgs, kernelTestImage, sobel, laplacian):
  n = 1

  # Display the original.
  plt.figure(figsize=(30,30 ))
  for i in range(0, n ):
      # Display original
      ax = plt.subplot(5, 1, 1)
      plt.imshow(kernelTestImage[i])
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)

  # If no filters used load it differently.
  if not sobel and not laplacian:
    # Display reconstruction
      ax = plt.subplot(5, 1, 1 + n)
      plt.imshow(decoded_imgs[0])
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
    

  # If either filter, load the imaes predicted from the filter models. 
  if sobel or laplacian:

      # Display reconstruction
    ax = plt.subplot(5, 1, 1 + n)
    plt.imshow(decoded_imgs[0][i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

      # Display reconstruction
    ax = plt.subplot(5, 1, 1 + n * 2)
    plt.imshow(decoded_imgs[1][i].squeeze(axis=2))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

        # Display reconstruction
    ax = plt.subplot(5, 1, 1 + n * 3)
    plt.imshow(decoded_imgs[2][i].squeeze(axis=2))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

  if laplacian:
    # Display reconstruction
    ax = plt.subplot(5, 1, 1 + n * 4)
    plt.imshow(decoded_imgs[3][i].squeeze(axis=2))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
      
  plt.show()

In [ ]:
# Get all the image names.
imageNames = loadAllImageNames()

# Get a test image.
testImage = getTestImage(imageNames)

# List for the different models.
models = []

# List to save the paths. 
locationList = []

# Add the needed paths.
if sobel:
  locationList.append(sobel_one_path)
  locationList.append(sobel_two_path)
elif laplacian:
  locationList.append(laplacian_one_path)
  locationList.append(laplacian_two_path)
  locationList.append(laplacian_three_path)

# If a filter is used, handle it.
if any([sobel,laplacian]):

  createModels(sobel, testImage)
  models = loadModels(sobel)

  # Create the numpy arrays. 
  print("Create the needed numpy array")
  createSaveNumpyArrays(locationList, imageNames, main_path, models)

# Set teh empty auto encoder.
autoEncoder = None

# Create the models as needed.
if deep:
  autoEncoder = getDeep(sobel, laplacian)
else:
  autoEncoder = getPaper(sobel, laplacian)

# Create the generators
trainGen = image_generator(BS,0,4500, sobel, laplacian, locationList,imageNames)
testGen = image_generator(BS, 4500,len(imageNames),sobel, laplacian, locationList,imageNames)

# Set the callback list.
callbackList = getCallbacks("/content/drive/MyDrive/Checkpoints/weights-improvements-{}-epoch{}".format(autoEncoderName,baseEpochs))

# Load auto encoder if need be.
if loadNameFitting is not None:
  autoEncoder = load_model(loadNameFitting)

trainImageCount = 4500
validationCount = len(imageNames) -4500

# Fit the algorithm. 
fitted = autoEncoder.fit(x = trainGen, steps_per_epoch= trainImageCount // BS, validation_data= testGen, validation_steps= validationCount // BS, epochs= EPOCHS, verbose = 1,callbacks = callbackList)

# Save the auto encoder. 
autoEncoder.save("/content/drive/MyDrive/ModelsRun2/{}_epoch{}".format(autoEncoderName,baseEpochs + EPOCHS))

# Run the actual model to test if it runs properly. 
decoded_img = autoEncoder.predict(testImage)
displayImage(decoded_img,testImage, sobel, laplacian)

# Save the losses.
saveLosses(fitted, autoEncoderName, baseEpochs + EPOCHS, any([sobel,laplacian]))



In [ ]:
# Load the auto encoder and do the predictions. 
autoEncoder = load_model(loadNamePrediction)
getPrediction(autoEncoder, main_path,imageNames, any([sobel,laplacian]))

In [ ]:
!zip -r deepWithoutFiltersEpoch100.zip /content/Predicted

In [ ]:
!cp /content/deepWithoutFiltersEpoch100.zip /content/drive/MyDrive/GeneratedImagesEpoch100/deepWithoutFiltersFacesEpoch100.zip